In [26]:
import os
import sys
import nglview
import ipywidgets

def showlog(file_path='log.out', n_lines=20, delete=True):
    with open(file_path) as f:
        cont=1
        for line in f:
            if cont > n_lines: break
            print(line, end='')
            cont +=1
    if delete:
        os.remove(file_path)


In [27]:
#Download PDB

#Import module
from biobb_io.mmb_api.pdb import MmbPdb
# Create prop dict and inputs/outputs
prop = {'pdb_code':'1ubq'}
downloaded_pdb_path = '1ubq.pdb'
#Create and launch bb
MmbPdb(output_pdb_path=downloaded_pdb_path, properties=prop).launch()
#Show log and protein
showlog()
nglview.show_file(downloaded_pdb_path)

2018-11-18 22:58:27,637 [MainThread  ] [INFO ]  
Downloading:
pdb_code: 1ubq
from: http://mmb.irbbarcelona.org/api/pdb/1ubq/coords/?filter=/1&group=ATOM
2018-11-18 22:58:28,031 [MainThread  ] [INFO ]  
Writting: 1ubq
to: /Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/1ubq.pdb


NGLWidget()

In [28]:
#Check & Fix PDB
#Import module
from biobb_model.model.fix_side_chain import FixSideChain
# Create prop dict and inputs/outputs
fixed_pdb_path = '1ubq_fixed.pdb'
#Create and launch bb
FixSideChain(input_pdb_path=downloaded_pdb_path, output_pdb_path=fixed_pdb_path, properties={}).launch()
#Show log and protein
showlog()
nglview.show_file(fixed_pdb_path)

=                     MDWeb structure checking utility                        =
=                      A. Hospital, J.L. Gelpi 2018                           =

Running fixside. Options: --fix All 
1ubq.pdb
Structure 1ubq.pdb loaded
Title: 
Experimental method: unknown
Resolution: 0.0 A

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  76
 Num. residues with ins. codes:  0
 Num. HETATM residues:  0
 Num. ligands or modified residues:  0
 Num. water mol.:  0
 Num. atoms:  602


NGLWidget()

In [29]:
#Mutate protein
# Import module to test
from biobb_model.model.mutate import Mutate
# Create prop dict and inputs/outputs
prop = {'mutation_list': 'Val26Ala'}
mutated_pdb_path = '1ubq_mutated.pdb'
#Create and launch bb
Mutate(input_pdb_path=fixed_pdb_path,output_pdb_path=mutated_pdb_path, properties=prop).launch()
#Show log and protein
showlog()
view1 = nglview.show_file(fixed_pdb_path)
view1.add_representation(repr_type='ball+stick', selection='26:A')
view2 = nglview.show_file(mutated_pdb_path)
view2.add_representation(repr_type='ball+stick', selection='26:A')
ipywidgets.VBox([view1, view2])

=                     MDWeb structure checking utility                        =
=                      A. Hospital, J.L. Gelpi 2018                           =

Running mutateside. Options: --mut Val26Ala 
1ubq_fixed.pdb
Structure 1ubq_fixed.pdb loaded
Title: 
Experimental method: unknown
Resolution: 0.0 A

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  76
 Num. residues with ins. codes:  0
 Num. HETATM residues:  0
 Num. ligands or modified residues:  0
 Num. water mol.:  0
 Num. atoms:  602


In [30]:
#Create system topology
# Import module to test
from biobb_md.gromacs.pdb2gmx import Pdb2gmx
# Create prop dict and inputs/outputs
output_pdb2gmx_gro_path = '1ubq_pdb2gmx.gro'
output_pdb2gmx_top_zip_path = '1ubq_pdb2gmx_top.zip'
#Create and launch bb
Pdb2gmx(input_pdb_path=mutated_pdb_path, output_gro_path=output_pdb2gmx_gro_path, output_top_zip_path=output_pdb2gmx_top_zip_path,  properties=prop).launch()
#Show log and protein
showlog()
nglview.show_file(output_pdb2gmx_gro_path)

2018-11-18 22:58:34,423 [MainThread  ] [INFO ]  gmx pdb2gmx -f 1ubq_mutated.pdb -o 1ubq_pdb2gmx.gro -p p2g.top -water spce -ff amber99sb-ildn -i p2g.itp

2018-11-18 22:58:34,425 [MainThread  ] [INFO ]  Exit code 0

2018-11-18 22:58:34,425 [MainThread  ] [INFO ]  b"\nUsing the Amber99sb-ildn force field in directory amber99sb-ildn.ff\n\nReading 1ubq_mutated.pdb...\nRead 600 atoms\nAnalyzing pdb file\nSplitting chemical chains based on TER records or chain id changing.\nThere are 1 chains and 0 blocks of water and 76 residues with 600 atoms\n\n  chain  #res #atoms\n  1 'A'    76    600  \n\nReading residue database... (amber99sb-ildn)\nProcessing chain 1 'A' (600 atoms, 76 residues)\nIdentified residue MET1 as a starting terminus.\nIdentified residue GLY76 as a ending terminus.\nChecking for duplicate atoms....\nGenerating any missing hydrogen atoms and/or adding termini.\nNow there are 76 residues with 1225 atoms\n\t\t--------- PLEASE NOTE ------------\nYou have successfully generated a

NGLWidget()

In [31]:
# Import module to test
from biobb_md.gromacs.editconf import Editconf
# Create prop dict and inputs/outputs
output_editconf_gro_path = '1ubq_editconf.gro'
#Create and launch bb
Editconf(input_gro_path=output_pdb2gmx_gro_path, output_gro_path=output_editconf_gro_path, properties={}).launch()
#Show log and protein
showlog()
nglview.show_file(output_editconf_gro_path)

2018-11-18 22:58:36,100 [MainThread  ] [INFO ]  Centering molecule in the box.
2018-11-18 22:58:36,100 [MainThread  ] [INFO ]  Distance of the box to molecule: 1.0
2018-11-18 22:58:36,101 [MainThread  ] [INFO ]  Box type: cubic
2018-11-18 22:58:36,120 [MainThread  ] [INFO ]  gmx editconf -f 1ubq_pdb2gmx.gro -o 1ubq_editconf.gro -d 1.0 -bt cubic -c

2018-11-18 22:58:36,122 [MainThread  ] [INFO ]  Exit code 0

2018-11-18 22:58:36,123 [MainThread  ] [INFO ]  b'Read 1225 atoms\nVolume: 36.3758 nm^3, corresponds to roughly 16300 electrons\nNo velocities found\n    system size :  3.069  3.238  3.662 (nm)\n    diameter    :  4.491               (nm)\n    center      :  3.025  2.902  1.539 (nm)\n    box vectors :  3.069  3.237  3.662 (nm)\n    box angles  :  90.00  90.00  90.00 (degrees)\n    box volume  :  36.38               (nm^3)\n    shift       :  0.221  0.344  1.707 (nm)\nnew center      :  3.246  3.246  3.246 (nm)\nnew box vectors :  6.491  6.491  6.491 (nm)\nnew box angles  :  90.00  

NGLWidget()

In [32]:
# Solvate: Fill the box with water molecules
from biobb_md.gromacs.solvate import Solvate
# Create prop dict and inputs/outputs
output_solvate_gro_path = '1ubq_solvate.gro'
output_solvate_top_zip_path = '1ubq_solvate_top.zip'
#Create and launch bb
Solvate(input_solute_gro_path=output_editconf_gro_path, output_gro_path=output_solvate_gro_path, input_top_zip_path=output_pdb2gmx_top_zip_path, output_top_zip_path=output_solvate_top_zip_path,  properties={}).launch()
#Show log and protein
showlog()
view = nglview.show_file(output_solvate_gro_path)
view.add_representation(repr_type='ball+stick', selection='SOL')
view

2018-11-18 22:58:37,168 [MainThread  ] [INFO ]  Extracting: /Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/1ubq_pdb2gmx_top.zip
2018-11-18 22:58:37,168 [MainThread  ] [INFO ]  to:
2018-11-18 22:58:37,169 [MainThread  ] [INFO ]  ['/Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/p2g.itp', '/Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/p2g.top', '/Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/grompp.top']
2018-11-18 22:58:37,170 [MainThread  ] [INFO ]  Moving: /Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/p2g.top to: /Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/solvate.top
2018-11-18 22:58:37,345 [MainThread  ] [INFO ]  gmx solvate -cp 1ubq_editconf.gro -cs spc216.gro -o 1ubq_solvate.gro -p solvate.top

2018-11-18 22:58:37,347 [MainThread  ] [INFO ]  Exit code 0

2018-11-18 22:58:37,348 [MainThread  ] [INFO ]  b'\nWARNING: Masses and atom

NGLWidget()

In [33]:
# Grompp: Creating portable binary run file for ion generation
from biobb_md.gromacs.grompp import Grompp
# Create prop dict and inputs/outputs
prop = {'mdp':{'type': 'minimization', 'nsteps':'5000'}}
output_gppion_tpr_path = '1ubq_gppion.tpr'
#Create and launch bb
Grompp(input_gro_path=output_solvate_gro_path, input_top_zip_path=output_solvate_top_zip_path, output_tpr_path=output_gppion_tpr_path,  properties=prop).launch()
#Show log and protein
showlog()

2018-11-18 22:58:39,516 [MainThread  ] [INFO ]  Extracting: /Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/1ubq_solvate_top.zip
2018-11-18 22:58:39,517 [MainThread  ] [INFO ]  to:
2018-11-18 22:58:39,518 [MainThread  ] [INFO ]  ['/Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/p2g.itp', '/Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/solvate.top', '/Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/grompp.top']
2018-11-18 22:58:39,519 [MainThread  ] [INFO ]  Moving: /Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/solvate.top to: /Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/grompp.top
2018-11-18 22:58:39,715 [MainThread  ] [INFO ]  gmx grompp -f grompp.mdp -c 1ubq_solvate.gro -r 1ubq_solvate.gro -p grompp.top -o 1ubq_gppion.tpr -maxwarn 10

2018-11-18 22:58:39,717 [MainThread  ] [INFO ]  Exit code 0

2018-11-18 22:58:39,718 [MainThread  ] [INF

In [34]:
# Genion: Adding ions to reach a 0.05 nm concentration
from biobb_md.gromacs.genion import Genion
# Create prop dict and inputs/outputs
output_genion_gro_path = '1ubq_genion.gro'
output_genion_top_zip_path = '1ubq_genion_top.zip'
#Create and launch bb
Genion(input_tpr_path=output_gppion_tpr_path, output_gro_path=output_genion_gro_path, input_top_zip_path=output_solvate_top_zip_path, output_top_zip_path=output_genion_top_zip_path, properties={}).launch()
#Show log and protein
showlog()
view = nglview.show_file(output_genion_gro_path)
view.add_representation(repr_type='ball+stick', selection='NA')
view.add_representation(repr_type='ball+stick', selection='CL')
view

2018-11-18 22:58:41,225 [MainThread  ] [INFO ]  To reach up 0.05 mol/litre concentration
2018-11-18 22:58:41,231 [MainThread  ] [INFO ]  Extracting: /Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/1ubq_solvate_top.zip
2018-11-18 22:58:41,232 [MainThread  ] [INFO ]  to:
2018-11-18 22:58:41,233 [MainThread  ] [INFO ]  ['/Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/p2g.itp', '/Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/solvate.top', '/Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/grompp.top']
2018-11-18 22:58:41,234 [MainThread  ] [INFO ]  Moving: /Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/solvate.top to: /Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/gio.top
2018-11-18 22:58:41,321 [MainThread  ] [INFO ]  echo "SOL" | gmx genion -s 1ubq_gppion.tpr -o 1ubq_genion.gro -p gio.top -conc 0.05 -seed 1993

2018-11-18 22:58:41,322 [MainThre

NGLWidget()

In [35]:
# Grompp: Creating portable binary run file for mdrun
from biobb_md.gromacs.grompp import Grompp
# Create prop dict and inputs/outputs
prop = {'mdp':{'type': 'minimization', 'nsteps':'5000'}}
output_gppmin_tpr_path = '1ubq_gppmin.tpr'
#Create and launch bb
Grompp(input_gro_path=output_genion_gro_path, input_top_zip_path=output_genion_top_zip_path, output_tpr_path=output_gppmin_tpr_path,  properties=prop).launch()
#Show log and protein
showlog()

2018-11-18 22:58:43,300 [MainThread  ] [INFO ]  Extracting: /Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/1ubq_genion_top.zip
2018-11-18 22:58:43,301 [MainThread  ] [INFO ]  to:
2018-11-18 22:58:43,302 [MainThread  ] [INFO ]  ['/Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/p2g.itp', '/Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/gio.top', '/Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/grompp.top']
2018-11-18 22:58:43,303 [MainThread  ] [INFO ]  Moving: /Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/gio.top to: /Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/grompp.top
2018-11-18 22:58:43,509 [MainThread  ] [INFO ]  gmx grompp -f grompp.mdp -c 1ubq_genion.gro -r 1ubq_genion.gro -p grompp.top -o 1ubq_gppmin.tpr -maxwarn 10

2018-11-18 22:58:43,511 [MainThread  ] [INFO ]  Exit code 0

2018-11-18 22:58:43,511 [MainThread  ] [INFO ]  b'Anal

In [36]:
# Mdrun: Running minimization
from biobb_md.gromacs.mdrun import Mdrun
# Create prop dict and inputs/outputs
output_min_trr_path = '1ubq_min.trr'
output_min_gro_path = '1ubq_min.gro'
output_min_edr_path = '1ubq_min.edr'
output_min_log_path = '1ubq_min.log'
#Create and launch bb
Mdrun(input_tpr_path=output_gppmin_tpr_path, output_trr_path=output_min_trr_path, output_gro_path=output_min_gro_path, output_edr_path=output_min_edr_path, output_log_path=output_min_log_path, properties={}).launch()
#Show log and protein
showlog()

2018-11-18 22:58:56,231 [MainThread  ] [INFO ]  gmx mdrun -s 1ubq_gppmin.tpr -o 1ubq_min.trr -c 1ubq_min.gro -e 1ubq_min.edr -g 1ubq_min.log -nt 0

2018-11-18 22:58:56,233 [MainThread  ] [INFO ]  Exit code 0

2018-11-18 22:58:56,234 [MainThread  ] [INFO ]  b''


In [37]:
# Grompp: Creating portable binary run file for mdrun
from biobb_md.gromacs.grompp import Grompp
# Create prop dict and inputs/outputs
prop = {'mdp':{'type': 'nvt', 'nsteps':'5000'}}
output_gppnvt_tpr_path = '1ubq_gppnvt.tpr'
#Create and launch bb
Grompp(input_gro_path=output_min_gro_path, input_top_zip_path=output_genion_top_zip_path, output_tpr_path=output_gppnvt_tpr_path,  properties=prop).launch()
#Show log and protein
showlog()

2018-11-18 22:58:59,450 [MainThread  ] [INFO ]  Extracting: /Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/1ubq_genion_top.zip
2018-11-18 22:58:59,452 [MainThread  ] [INFO ]  to:
2018-11-18 22:58:59,453 [MainThread  ] [INFO ]  ['/Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/p2g.itp', '/Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/gio.top', '/Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/grompp.top']
2018-11-18 22:58:59,454 [MainThread  ] [INFO ]  Moving: /Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/gio.top to: /Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/grompp.top
2018-11-18 22:58:59,741 [MainThread  ] [INFO ]  gmx grompp -f grompp.mdp -c 1ubq_min.gro -r 1ubq_min.gro -p grompp.top -o 1ubq_gppnvt.tpr -maxwarn 10

2018-11-18 22:58:59,743 [MainThread  ] [INFO ]  Exit code 0

2018-11-18 22:58:59,744 [MainThread  ] [INFO ]  b'turning al

In [38]:
# Mdrun: Running minimization
from biobb_md.gromacs.mdrun import Mdrun
# Create prop dict and inputs/outputs
output_nvt_trr_path = '1ubq_nvt.trr'
output_nvt_gro_path = '1ubq_nvt.gro'
output_nvt_edr_path = '1ubq_nvt.edr'
output_nvt_log_path = '1ubq_nvt.log'
output_nvt_cpt_path = '1ubq_nvt.cpt'
#Create and launch bb
Mdrun(input_tpr_path=output_gppnvt_tpr_path, output_trr_path=output_nvt_trr_path, output_gro_path=output_nvt_gro_path, output_edr_path=output_nvt_edr_path, output_log_path=output_nvt_log_path, output_cpt_path=output_nvt_cpt_path, properties={}).launch()
#Show log and protein
showlog()

2018-11-18 23:00:14,955 [MainThread  ] [INFO ]  gmx mdrun -s 1ubq_gppnvt.tpr -o 1ubq_nvt.trr -c 1ubq_nvt.gro -e 1ubq_nvt.edr -g 1ubq_nvt.log -nt 0 -cpo 1ubq_nvt.cpt

2018-11-18 23:00:14,956 [MainThread  ] [INFO ]  Exit code 0

2018-11-18 23:00:14,957 [MainThread  ] [INFO ]  b''


In [39]:
# Grompp: Creating portable binary run file for mdrun
from biobb_md.gromacs.grompp import Grompp
# Create prop dict and inputs/outputs
prop = {'mdp':{'type': 'npt', 'nsteps':'5000'}}
output_gppnpt_tpr_path = '1ubq_gppnpt.tpr'
#Create and launch bb
Grompp(input_gro_path=output_nvt_gro_path, input_top_zip_path=output_genion_top_zip_path, output_tpr_path=output_gppnpt_tpr_path, input_cpt_path=output_nvt_cpt_path,  properties=prop).launch()
#Show log and protein
showlog()

2018-11-18 23:00:37,610 [MainThread  ] [INFO ]  Extracting: /Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/1ubq_genion_top.zip
2018-11-18 23:00:37,611 [MainThread  ] [INFO ]  to:
2018-11-18 23:00:37,613 [MainThread  ] [INFO ]  ['/Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/p2g.itp', '/Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/gio.top', '/Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/grompp.top']
2018-11-18 23:00:37,616 [MainThread  ] [INFO ]  Moving: /Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/gio.top to: /Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/grompp.top
2018-11-18 23:00:37,994 [MainThread  ] [INFO ]  gmx grompp -f grompp.mdp -c 1ubq_nvt.gro -r 1ubq_nvt.gro -p grompp.top -o 1ubq_gppnpt.tpr -maxwarn 10 -t 1ubq_nvt.cpt

2018-11-18 23:00:37,996 [MainThread  ] [INFO ]  Exit code 0

2018-11-18 23:00:37,997 [MainThread  ] [INFO

In [40]:
# Mdrun: Running minimization
from biobb_md.gromacs.mdrun import Mdrun
# Create prop dict and inputs/outputs
output_npt_trr_path = '1ubq_npt.trr'
output_npt_gro_path = '1ubq_npt.gro'
output_npt_edr_path = '1ubq_npt.edr'
output_npt_log_path = '1ubq_npt.log'
output_npt_cpt_path = '1ubq_npt.cpt'
#Create and launch bb
Mdrun(input_tpr_path=output_gppnpt_tpr_path, output_trr_path=output_npt_trr_path, output_gro_path=output_npt_gro_path, output_edr_path=output_npt_edr_path, output_log_path=output_npt_log_path, output_cpt_path=output_npt_cpt_path, properties={}).launch()
#Show log and protein
showlog()

2018-11-18 23:01:21,888 [MainThread  ] [INFO ]  gmx mdrun -s 1ubq_gppnpt.tpr -o 1ubq_npt.trr -c 1ubq_npt.gro -e 1ubq_npt.edr -g 1ubq_npt.log -nt 0 -cpo 1ubq_npt.cpt

2018-11-18 23:01:21,890 [MainThread  ] [INFO ]  Exit code 0

2018-11-18 23:01:21,891 [MainThread  ] [INFO ]  b''


In [42]:
# Grompp: Creating portable binary run file for mdrun
from biobb_md.gromacs.grompp import Grompp
# Create prop dict and inputs/outputs
prop = {'mdp':{'type': 'free', 'nsteps':'50000'}}
output_gppmd_tpr_path = '1ubq_gppmd.tpr'
#Create and launch bb
Grompp(input_gro_path=output_npt_gro_path, input_top_zip_path=output_genion_top_zip_path, output_tpr_path=output_gppmd_tpr_path, input_cpt_path=output_npt_cpt_path,  properties=prop).launch()
#Show log and protein
showlog()

2018-11-18 23:08:07,432 [MainThread  ] [INFO ]  Extracting: /Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/1ubq_genion_top.zip
2018-11-18 23:08:07,434 [MainThread  ] [INFO ]  to:
2018-11-18 23:08:07,435 [MainThread  ] [INFO ]  ['/Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/p2g.itp', '/Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/gio.top', '/Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/grompp.top']
2018-11-18 23:08:07,438 [MainThread  ] [INFO ]  Moving: /Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/gio.top to: /Users/pau/projects/biobb_workflows/biobb_workflows/ubiquitin/notebooks/grompp.top
2018-11-18 23:08:07,720 [MainThread  ] [INFO ]  gmx grompp -f grompp.mdp -c 1ubq_npt.gro -r 1ubq_npt.gro -p grompp.top -o 1ubq_gppmd.tpr -maxwarn 10 -t 1ubq_npt.cpt

2018-11-18 23:08:07,722 [MainThread  ] [INFO ]  Exit code 0

2018-11-18 23:08:07,723 [MainThread  ] [INFO 

In [41]:
# Mdrun: Running free dynamics
from biobb_md.gromacs.mdrun import Mdrun
# Create prop dict and inputs/outputs
output_md_trr_path = '1ubq_md.trr'
output_md_gro_path = '1ubq_md.gro'
output_md_edr_path = '1ubq_md.edr'
output_md_log_path = '1ubq_md.log'
output_md_cpt_path = '1ubq_md.cpt'
#Create and launch bb
Mdrun(input_tpr_path=output_gppmd_tpr_path, output_trr_path=output_md_trr_path, output_gro_path=output_md_gro_path, output_edr_path=output_md_edr_path, output_log_path=output_md_log_path, output_cpt_path=output_md_cpt_path, properties={}).launch()
#Show log and protein
showlog()
nglview.show_simpletraj(nglview.SimpletrajTrajectory(output_md_trr_path, output_md_gro_path), gui=True)

2018-11-18 23:08:07,191 [MainThread  ] [INFO ]  gmx mdrun -s 1ubq_gppmd.tpr -o 1ubq_md.trr -c 1ubq_md.gro -e 1ubq_md.edr -g 1ubq_md.log -nt 0 -cpo 1ubq_md.cpt

2018-11-18 23:08:07,201 [MainThread  ] [INFO ]  Exit code 0

2018-11-18 23:08:07,202 [MainThread  ] [INFO ]  b''


NGLWidget(count=11)

In [ ]:
#Clean the directory Remove everything
!find . ! -iname "*.ipynb" -exec rm -rf {} 2>/dev/null \; -print